In [1]:

# ==============================================================
# fix_data.py  ←  SELF-HEALING SCRIPT  (Airflow Friendly)
# ==============================================================

import sqlite3
import pandas as pd
import numpy as np
import logging


DB = r"E:\work\DEPI\graduation promax\data\warehouse_final\ivf_star_schema.db"
LOG = r"E:\work\DEPI\graduation promax\src\ETL\logs\repair_log.txt"
# لو validation نجح → ده يبقى = 1  | لو في مشاكل → = -1
STATUS_FLAG = -1     # 👈 أثناء الاختبار خليه -1

logging.basicConfig(
    filename=LOG,
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)


# ==============================================================
# STEP 1 → REPLACE NULL VALUES
# ==============================================================

def replace_nulls(df):
    """ استبدال NULL بـ 'None' """
    return df.fillna("None")


# ==============================================================
# STEP 2 → REMOVE DUPLICATES (But keep first)
# ==============================================================

def handle_duplicates(conn, table):
    """ يحذف الصفوف المتكررة مع الحفاظ على أول نسخة فقط """
    if table.startswith("sqlite_"):
        logging.info(f"Skipping system table: {table}")
        return

    df = pd.read_sql(f"SELECT * FROM {table}", conn)
    dup_count = df.duplicated().sum()

    if dup_count > 0:
        logging.warning(f"Found {dup_count} duplicates in {table} – Cleaning...")

        df_clean = df.drop_duplicates(keep="first")
        df_clean.to_sql(table, conn, if_exists="replace", index=False)

        print(f"✔ Duplicates fixed in {table}")
        logging.info(f"Duplicates removed ({dup_count}) in {table}")
    else:
        logging.info(f"No duplicates found in {table}")


# ==============================================================
# STEP 3 → FIX FOREIGN KEYS
# ==============================================================


def fix_foreign_keys(conn):
    """ يحاول يصلّح مشاكل العلاقات بين ال fact tables و ال dimensions """
    logging.warning("Fixing invalid FK relations...")




    # ============================================
    # 2) معالجة fact_transfer FK invalid cases
    # ============================================
    try:
        df = pd.read_sql("""
            SELECT ft.*
            FROM fact_transfer ft
            LEFT JOIN dim_time dt ON ft.transfer_time_fk = dt.time_id
            WHERE dt.time_id IS NULL
               AND ft.transfer_time_fk IS NOT NULL
        """, conn)

        if not df.empty:
            logging.warning(f"Fixing {len(df)} orphan records in fact_transfer")
            print(f"🛠 Fixing {len(df)} orphan records in fact_transfer...")

            time_df = pd.read_sql("SELECT time_id, full_date FROM dim_time", conn)
            date_to_id = dict(zip(time_df["full_date"], time_df["time_id"]))

            df["transfer_time_fk"] = df["transfer_time_fk"].fillna(0).astype(int)
            df["transfer_time_fk"] = df["transfer_time_fk"].map(lambda x: date_to_id.get(x, 0))

            conn.execute("DELETE FROM fact_transfer")  # نعيد ملء الجدول
            df.to_sql("fact_transfer", conn, if_exists="append", index=False)

            print("✔ FK fix applied for fact_transfer")
            logging.info("FK in fact_transfer repaired!")

    except Exception as e:
        logging.error(f"FK in fact_transfer ERROR: {e}")

    # ============================================
    # 3) معالجة fact_transfer_embryo لو fact_transfer اتظبط
    # ============================================
    try:
        existing_transfer = pd.read_sql("SELECT transfer_sk, case_id FROM fact_transfer", conn)
        if not existing_transfer.empty:
            print("🛠 Updating fact_transfer_embryo based on new transfer_sk...")

            df_emb = pd.read_sql("SELECT * FROM fact_transfer_embryo", conn)
            df_merge = df_emb.merge(existing_transfer, on="case_id", how="left")

            df_merge = df_merge[["transfer_sk", "embryo_id"]].drop_duplicates()
            conn.execute("DELETE FROM fact_transfer_embryo")
            df_merge.to_sql("fact_transfer_embryo", conn, if_exists="append", index=False)

            print("✔ fact_transfer_embryo fixed!")
            logging.info("fact_transfer_embryo FK fixed.")

    except Exception as e:
        logging.error(f"FK fix for fact_transfer_embryo ERROR: {e}")




    # ============================================
    # 1) معالجة التواريخ المفقودة في dim_time
    # ============================================
def fix_missing_time_fk(conn):
    """
    أي صف في fact_transfer عنده transfer_time_fk
    ومش لاقينه في dim_time → نخليه NULL.
    """
    try:
        # كام صف FK بتاعه مش موجود في dim_time؟
        df = pd.read_sql("""
            SELECT COUNT(*) AS cnt
            FROM fact_transfer ft
            LEFT JOIN dim_time dt 
                ON ft.transfer_time_fk = dt.time_id
            WHERE ft.transfer_time_fk IS NOT NULL
              AND dt.time_id IS NULL
        """, conn)

        missing_cnt = int(df["cnt"].iloc[0])

        if missing_cnt == 0:
            print("💚 No invalid time_fk in fact_transfer")
            logging.info("No invalid time_fk in fact_transfer")
            return

        print(f"🛠 Setting {missing_cnt} invalid time_fk to NULL in fact_transfer...")

        conn.execute("""
            UPDATE fact_transfer
            SET transfer_time_fk = NULL
            WHERE transfer_time_fk IS NOT NULL
              AND transfer_time_fk NOT IN (SELECT time_id FROM dim_time)
        """)
        conn.commit()

        print("✔ Invalid time_fk set to NULL")
        logging.info(f"Fixed {missing_cnt} invalid time_fk in fact_transfer (set to NULL).")

    except Exception as e:
        logging.error(f"Time FK Fix Error: {e}")
        print(f"❌ ERROR in time_fk fix: {e}")


def fix_missing_cycle_time_fk(conn):
    """
    أي صف في fact_ivf_cycle عنده cycle_start_time_id
    ومش موجود في dim_time → نخليه NULL.
    """
    try:
        df = pd.read_sql("""
            SELECT COUNT(*) AS cnt
            FROM fact_ivf_cycle fc
            LEFT JOIN dim_time dt 
                ON fc.cycle_start_time_id = dt.time_id
            WHERE fc.cycle_start_time_id IS NOT NULL
              AND dt.time_id IS NULL
        """, conn)

        missing_cnt = int(df["cnt"].iloc[0])

        if missing_cnt == 0:
            print("💚 No invalid cycle_start_time_id in fact_ivf_cycle")
            logging.info("No invalid cycle_start_time_id in fact_ivf_cycle")
            return

        print(f"🛠 Setting {missing_cnt} invalid cycle_start_time_id to NULL in fact_ivf_cycle...")

        conn.execute("""
            UPDATE fact_ivf_cycle
            SET cycle_start_time_id = NULL
            WHERE cycle_start_time_id IS NOT NULL
              AND cycle_start_time_id NOT IN (SELECT time_id FROM dim_time)
        """)
        conn.commit()

        print("✔ Invalid cycle_start_time_id set to NULL")
        logging.info(f"Fixed {missing_cnt} invalid cycle_start_time_id in fact_ivf_cycle (set to NULL).")

    except Exception as e:
        logging.error(f"Cycle Time FK Fix Error: {e}")
        print(f"❌ ERROR in cycle time FK fix: {e}")

# ==============================================================
# MAIN FUNCTION → RUN FIXES
# ==============================================================
def run_fixes():
    if STATUS_FLAG == 1:
        print("💚 Database is CLEAN — No fixes needed")
        return

    print("\n⚠ Problems detected — APPLYING FIXES...\n")
    conn = sqlite3.connect(DB)

    # 1) NULL → 'None' (زي ما عاملين)
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
    for tbl in tables["name"]:
        if tbl.startswith("sqlite_"):
            continue
        df = pd.read_sql(f"SELECT * FROM {tbl}", conn)
        df = replace_nulls(df)
        df.to_sql(tbl, conn, if_exists="replace", index=False)

    # 2) FIX DUPLICATES
    for tbl in tables["name"]:
        handle_duplicates(conn, tbl)

    # 3) FIX FK RELATIONS (اللي عندك بالفعل)
    fix_foreign_keys(conn)

    # 4) 🆕 FIX MISSING TIME FK BY SETTING NULL
    fix_missing_time_fk(conn)
    fix_missing_cycle_time_fk(conn)


    conn.close()
    print("\n✨ Fixing DONE — Database is NOW CLEAN!")
    logging.info("All fixes applied successfully")


if __name__ == "__main__":
    run_fixes()



⚠ Problems detected — APPLYING FIXES...

🛠 Fixing 1 orphan records in fact_transfer...
🛠 Updating fact_transfer_embryo based on new transfer_sk...
🛠 Setting 1 invalid time_fk to NULL in fact_transfer...
✔ Invalid time_fk set to NULL
🛠 Setting 1 invalid cycle_start_time_id to NULL in fact_ivf_cycle...
✔ Invalid cycle_start_time_id set to NULL

✨ Fixing DONE — Database is NOW CLEAN!
